# Anotador Proteico – InterProScan para Uma Única Sequência

Este notebook recebe **uma sequência proteica** (como string) e utiliza o **InterProScan (EBI)** para:
- buscar domínios e famílias em múltiplos bancos;
- fazer **classificação por confiança** com base no número de bancos de domínios que concordam;
- gerar **visualizações** simples;
- produzir um **relatório final** em arquivo de texto.

## 0. Configuração Inicial do Ambiente

Execute esta seção primeiro para configurar o ambiente (por exemplo, no Google Colab).

Serão instalados/garantidos os principais pacotes Python usados ao longo do notebook:
- **biopython** (não é estritamente obrigatório aqui, mas útil para usos futuros);
- **pandas**: organização das tabelas de domínios;
- **matplotlib** e **seaborn**: geração de gráficos;
- **requests**: comunicação com a API REST do InterProScan (EBI).

In [ ]:
# Instalar dependências necessárias (ajustado para Google Colab ou ambiente similar)
print("=" * 70)
print("CONFIGURANDO AMBIENTE PYTHON")
print("=" * 70)

print("\n📦 Instalando/atualizando pacotes Python...")
!pip install -q biopython pandas matplotlib seaborn requests

print("\n✅ Ambiente configurado com sucesso!")
print("=" * 70)

## 1. Informar a Sequência Proteica

Edite a célula de código abaixo para definir:
- um identificador para a proteína (`seq_id`);
- a sequência de aminoácidos completa (`sequence`), em código de uma letra.

A sequência deve ser apenas letras de aminoácidos (A, C, D, E, ...), sem quebras de linha.

In [ ]:
# Defina aqui a sua sequência proteica
seq_id = "MINHA_PROTEINA"  # Identificador da sequência

# INSIRA a sequência de aminoácidos abaixo (string contínua, sem espaços/linhas)
sequence = ""  # Exemplo: "MKLSKNQNLLDIYGVTGSGKSTLLRCVNLIEKDSG..."

if not sequence:
    print("⚠️  Atenção: a variável 'sequence' está vazia. Edite esta célula e insira a sua sequência.")
else:
    print(f"✅ Sequência definida para {seq_id} com {len(sequence)} aminoácidos.")

## 2. Configuração do e-mail para o InterProScan

A API REST do **InterProScan (EBI)** pede um e-mail de contato válido.
Como este notebook pode ser reutilizado por diferentes pessoas, o e-mail **não** fica fixo no código.

Antes de rodar a predição de domínios, **edite a célula de código abaixo** e insira o seu e-mail de contato onde está indicado (`INSIRA email aqui`).

Use, de preferência, um e-mail institucional ou acadêmico.

In [ ]:
# E-mail de contato para uso da API do InterProScan
user_email = ""  # INSIRA email aqui (por exemplo: "seu.nome@instituicao.br")

if not user_email:
    print("⚠️  Atenção: a variável 'user_email' está vazia. Edite esta célula e insira um e-mail válido antes de executar as buscas.")
else:
    print(f"✅ E-mail configurado: {user_email}")

## 3. Busca de Domínios via InterProScan (EBI)

Nesta etapa, usamos a **API REST do InterProScan (iprscan5)** para consultar todos os bancos disponíveis em uma única chamada, incluindo:
- **Pfam, SMART, PROSITE, PANTHER, PRINTS, PIRSF/PIRSR, TIGRFAMs, CDD, SFLD** (domínios/famílias funcionais);
- **Gene3D, SUPERFAMILY** (domínios/famílias estruturais);
- **PHOBIUS, TMHMM, SIGNALP** e outros preditores de topologia/localização.

Os resultados são salvos em um DataFrame (`df_domains`) e em um arquivo CSV consolidado para uso posterior.

In [ ]:
import requests
import time
import json
import os
import pandas as pd
from collections import Counter

# Garantir pasta de saída
os.makedirs('outputs', exist_ok=True)

def search_all_databases(sequence, seq_id, email, timeout=600):
    """Busca domínios em TODOS os bancos de uma vez (InterProScan completo)"""
    try:
        if not email:
            print("⚠️  Nenhum e-mail fornecido. Interrompendo requisição.")
            return []

        print(f"   Enviando sequência para InterProScan ({seq_id})...", end="", flush=True)
        submit_url = "https://www.ebi.ac.uk/Tools/services/rest/iprscan5/run"
        params = {
            'email': email,
            'sequence': sequence,
            'title': seq_id,
            'goterms': 'false',
            'pathways': 'false'
        }
        response = requests.post(submit_url, data=params, timeout=30)
        if response.status_code != 200:
            print(f" ⚪ [HTTP {response.status_code}]", end="")
            return []

        job_id = response.text.strip()
        poll_interval = 5
        max_attempts = timeout // poll_interval

        for attempt in range(max_attempts):
            time.sleep(poll_interval)
            try:
                status_resp = requests.get(
                    f"https://www.ebi.ac.uk/Tools/services/rest/iprscan5/status/{job_id}",
                    timeout=10
                )
            except requests.Timeout:
                print(" ⚠️ [Timeout ao checar status]", end="")
                continue

            if status_resp.status_code != 200:
                print(f" ⚪ [HTTP {status_resp.status_code} ao checar status]", end="")
                continue

            status = status_resp.text.strip()

            if status == 'FINISHED':
                try:
                    result = requests.get(
                        f"https://www.ebi.ac.uk/Tools/services/rest/iprscan5/result/{job_id}/json",
                        timeout=60
                    )
                except requests.Timeout:
                    print(" ⚠️ [Timeout ao baixar resultado]", end="")
                    return []

                if result.status_code == 200:
                    data = result.json()
                    domains = []
                    for resultset in data.get('results', []):
                        for match in resultset.get('matches', []):
                            sig = match.get('signature', {})
                            sig_lib = sig.get('signatureLibraryRelease', {})
                            entry = sig.get('entry') or {}
                            locations = match.get('locations', [])

                            if locations:
                                for loc in locations:
                                    domains.append({
                                        'seq_id': seq_id,
                                        'database': sig_lib.get('library', 'UNKNOWN'),
                                        'database_version': sig_lib.get('version', ''),
                                        'accession': sig.get('accession', ''),
                                        'name': sig.get('name', ''),
                                        'description': sig.get('description', ''),
                                        'type': sig.get('type', ''),
                                        'model_ac': match.get('model-ac', ''),
                                        'evalue': match.get('evalue', None),
                                        'score': match.get('score', None),
                                        'start': loc.get('start', None),
                                        'end': loc.get('end', None),
                                        'length': loc.get('end', 0) - loc.get('start', 0) + 1 if loc.get('start') and loc.get('end') else None,
                                        'location_evalue': loc.get('evalue', None),
                                        'location_score': loc.get('score', None),
                                        'hmm_start': loc.get('hmmStart', None),
                                        'hmm_end': loc.get('hmmEnd', None),
                                        'hmm_length': loc.get('hmmLength', None),
                                        'hmm_bounds': loc.get('hmmBounds', ''),
                                        'envelope_start': loc.get('envelopeStart', None),
                                        'envelope_end': loc.get('envelopeEnd', None),
                                        'representative': loc.get('representative', False),
                                        'post_processed': loc.get('postProcessed', False),
                                        'interpro_accession': entry.get('accession', ''),
                                        'interpro_name': entry.get('name', ''),
                                        'interpro_description': entry.get('description', ''),
                                        'interpro_type': entry.get('type', '')
                                    })
                            else:
                                domains.append({
                                    'seq_id': seq_id,
                                    'database': sig_lib.get('library', 'UNKNOWN'),
                                    'database_version': sig_lib.get('version', ''),
                                    'accession': sig.get('accession', ''),
                                    'name': sig.get('name', ''),
                                    'description': sig.get('description', ''),
                                    'type': sig.get('type', ''),
                                    'model_ac': match.get('model-ac', ''),
                                    'evalue': match.get('evalue', None),
                                    'score': match.get('score', None),
                                    'start': None,
                                    'end': None,
                                    'length': None,
                                    'location_evalue': None,
                                    'location_score': None,
                                    'hmm_start': None,
                                    'hmm_end': None,
                                    'hmm_length': None,
                                    'hmm_bounds': '',
                                    'envelope_start': None,
                                    'envelope_end': None,
                                    'representative': False,
                                    'post_processed': False,
                                    'interpro_accession': entry.get('accession', ''),
                                    'interpro_name': entry.get('name', ''),
                                    'interpro_description': entry.get('description', ''),
                                    'interpro_type': entry.get('type', '')
                                })

                    if domains:
                        print(f" ✅ ({len(domains)} domínios)", end="")
                    else:
                        print(" ⚪ Nenhum domínio encontrado.", end="")
                    return domains
                else:
                    print(f" ⚪ [HTTP {result.status_code}]", end="")
                    return []

            elif status in ['FAILED', 'ERROR', 'NOT_FOUND']:
                print(f" ⚪ [Status {status}]", end="")
                return []
            elif status == 'RUNNING':
                continue
            else:
                print(f" ⚠️ [Status desconhecido: {status}]", end="")
                continue

        elapsed_time = max_attempts * poll_interval
        print(f" ⏱ [Timeout após {elapsed_time}s]", end="")
        return []

    except requests.Timeout:
        print(" ⚠️ [Timeout HTTP]", end="")
        return []
    except Exception as e:
        print(f" ⚠️ [{type(e).__name__}: {str(e)[:50]}]", end="")
        return []

# Executar busca de domínios para a sequência informada
print("=" * 70)
print("BUSCA DE DOMÍNIOS PARA UMA ÚNICA SEQUÊNCIA")
print("=" * 70)

if not sequence:
    print("❌ A sequência está vazia. Volte à seção 1 e insira a sequência.")
elif not user_email:
    print("❌ O e-mail não foi configurado. Volte à seção 2 e insira um e-mail válido.")
else:
    start_time = time.time()
    domains = search_all_databases(sequence, seq_id, user_email, timeout=600)
    elapsed = time.time() - start_time
    print(f"\n\n⏱ Tempo total: {elapsed:.2f}s")

    if domains:
        df_domains = pd.DataFrame(domains)
        out_path = 'outputs/output_all_domains_single_sequence.csv'
        df_domains.to_csv(out_path, index=False)
        print(f"✅ Domínios salvos em: {out_path}")
        print(f"📊 Total de anotações: {len(df_domains)}")
        print(f"🗃️ Bancos distintos: {df_domains['database'].nunique()}")
    else:
        df_domains = pd.DataFrame()
        print("⚪ Nenhum domínio retornado pelo InterProScan.")

## 4. Classificação por Confiança e Integração

Aqui, classificamos a proteína com base em **quantos bancos de domínios diferentes** (funcionais/estruturais) encontraram domínios nela:
- **Alta confiança**: domínios em ≥5 bancos;
- **Média confiança**: domínios em 3–4 bancos;
- **Baixa confiança**: domínios em 1–2 bancos;
- **Sem domínios**: nenhum banco de domínios encontrou domínios.

Também sumarizamos separadamente as anotações de **topologia** (PHOBIUS, TMHMM, SIGNALP).

In [ ]:
# Classificação por confiança para uma única sequência
if 'df_domains' in globals() and not df_domains.empty:
    print("=" * 70)
    print("CLASSIFICAÇÃO POR CONFIANÇA – SEQUÊNCIA ÚNICA")
    print("=" * 70)

    FUNCTIONAL_DOMAINS = ['PFAM', 'SMART', 'PROSITE', 'PANTHER', 'PRINTS',
                          'PIRSF', 'PIRSR', 'HAMAP', 'TIGERFAMS', 'SFLD', 'CDD']
    STRUCTURAL_DOMAINS = ['GENE3D', 'SUPERFAMILY']
    DOMAIN_DATABASES = FUNCTIONAL_DOMAINS + STRUCTURAL_DOMAINS

    TOPOLOGY = ['PHOBIUS', 'TMHMM', 'SIGNALP_EUK', 'SIGNALP_GRAM_POSITIVE',
                'SIGNALP_GRAM_NEGATIVE']

    df_domains_only = df_domains[df_domains['database'].isin(DOMAIN_DATABASES)]
    df_topology = df_domains[df_domains['database'].isin(TOPOLOGY)]

    if not df_domains_only.empty:
        unique_dbs = df_domains_only['database'].unique()
        num_dbs = len(unique_dbs)
        unique_domains = df_domains_only['name'].dropna().unique()

        if num_dbs >= 5:
            confidence = 'Alta confiança (≥5 DBs)'
            confidence_level = 'Alta'
        elif num_dbs >= 3:
            confidence = 'Média confiança (3-4 DBs)'
            confidence_level = 'Média'
        elif num_dbs >= 1:
            confidence = 'Baixa confiança (1-2 DBs)'
            confidence_level = 'Baixa'
        else:
            confidence = 'SEM DOMÍNIOS'
            confidence_level = 'Nenhum'

        print(f"\n📊 Bancos de DOMÍNIOS que anotaram a proteína {seq_id}:")
        print(f"   Bancos: {', '.join(sorted(unique_dbs))}")
        print(f"   Número de bancos: {num_dbs}")
        print(f"   Número de domínios únicos: {len(unique_domains)}")
        print(f"   Nível de confiança: {confidence} (classe: {confidence_level})")
    else:
        confidence = 'SEM DOMÍNIOS'
        confidence_level = 'Nenhum'
        print(f"\n⚪ Nenhum domínio em bancos funcionais/estruturais foi encontrado para {seq_id}.")

    # Topologia
    has_tm = False
    has_signal = False
    if not df_topology.empty:
        has_tm = df_topology['name'].str.contains('Transmembrane', case=False, na=False).any()
        has_signal = df_topology['name'].str.contains('Signal', case=False, na=False).any()

        print("\n📌 Anotações de TOPOLOGIA (PHOBIUS/TMHMM/SIGNALP):")
        for db in df_topology['database'].unique():
            count = len(df_topology[df_topology['database'] == db])
            print(f"   - {db}: {count} anotações")
    else:
        print("\n📌 Nenhuma anotação de topologia encontrada.")

    # Guardar resumo em um dicionário para uso em outras células
    single_summary = {
        'seq_id': seq_id,
        'num_domain_databases': int(num_dbs) if 'num_dbs' in locals() else 0,
        'confidence': confidence,
        'confidence_level': confidence_level,
        'has_transmembrane': bool(has_tm),
        'has_signal_peptide': bool(has_signal),
        'num_domains': int(len(unique_domains)) if 'unique_domains' in locals() else 0
    }
else:
    print("❌ DataFrame 'df_domains' vazio ou não definido. Execute a seção 3 primeiro.")

## 5. Visualização e Análise de Domínios

Geração de gráficos simples para a sequência analisada, incluindo:
- distribuição das anotações por banco de dados;
- destaque das categorias de bancos (funcionais, estruturais, topologia);
- indicação visual de presença/ausência de características topológicas.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

if 'df_domains' in globals() and not df_domains.empty:
    print("=" * 70)
    print("VISUALIZAÇÃO DE DOMÍNIOS – SEQUÊNCIA ÚNICA")
    print("=" * 70)

    FUNCTIONAL_DOMAINS = ['PFAM', 'SMART', 'PROSITE', 'PANTHER', 'PRINTS',
                          'PIRSF', 'PIRSR', 'HAMAP', 'TIGERFAMS', 'SFLD', 'CDD']
    STRUCTURAL_DOMAINS = ['GENE3D', 'SUPERFAMILY']
    TOPOLOGY = ['PHOBIUS', 'TMHMM', 'SIGNALP_EUK', 'SIGNALP_GRAM_POSITIVE',
                'SIGNALP_GRAM_NEGATIVE']

    df_domains['category'] = 'Outros'
    df_domains.loc[df_domains['database'].isin(FUNCTIONAL_DOMAINS), 'category'] = 'Funcional'
    df_domains.loc[df_domains['database'].isin(STRUCTURAL_DOMAINS), 'category'] = 'Estrutural'
    df_domains.loc[df_domains['database'].isin(TOPOLOGY), 'category'] = 'Topologia'

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Gráfico 1: contagem por banco
    db_counts = df_domains['database'].value_counts().sort_values(ascending=False)
    sns.barplot(x=db_counts.values, y=db_counts.index, ax=axes[0], palette='viridis')
    axes[0].set_xlabel('Número de anotações')
    axes[0].set_ylabel('Banco de dados')
    axes[0].set_title('Anotações por banco de dados')

    # Gráfico 2: categorias de bancos
    cat_counts = df_domains['category'].value_counts()
    axes[1].pie(cat_counts.values, labels=cat_counts.index, autopct='%1.1f%%', startangle=90)
    axes[1].set_title('Categorias de bancos (Funcional/Estrutural/Topologia)')

    plt.suptitle(f'Análise de domínios para {seq_id}', fontsize=14, fontweight='bold')
    out_fig = 'outputs/single_sequence_domain_analysis.png'
    plt.savefig(out_fig, dpi=300, bbox_inches='tight')
    print(f"✅ Figura salva em: {out_fig}")
    plt.show()
else:
    print("❌ Dados insuficientes para visualização. Execute as seções 3 e 4 primeiro.")

## 6. Relatório Final

Gera um relatório de texto com:
- resumo da confiança na anotação de domínios;
- lista de domínios encontrados;
- observações sobre características topológicas (quando presentes).

O relatório é salvo na pasta `outputs/`.

In [ ]:
# Geração do relatório final em texto
if 'df_domains' in globals() and not df_domains.empty and 'single_summary' in globals():
    report_path = 'outputs/relatorio_dominios_single_sequence.txt'
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write("="*70 + "\n")
        f.write("RELATÓRIO DE DOMÍNIOS – SEQUÊNCIA ÚNICA\n")
        f.write("="*70 + "\n\n")

        f.write(f"Sequência: {single_summary['seq_id']}\n")
        f.write(f"Número de bancos de domínios: {single_summary['num_domain_databases']}\n")
        f.write(f"Número de domínios únicos: {single_summary['num_domains']}\n")
        f.write(f"Nível de confiança: {single_summary['confidence']} (classe: {single_summary['confidence_level']})\n\n")

        f.write("DOMÍNIOS ENCONTRADOS:\n")
        f.write("-"*70 + "\n")
        for acc, name in sorted(set(zip(df_domains['accession'], df_domains['name']))):
            f.write(f"{acc}: {name}\n")

        f.write("\nCARACTERÍSTICAS TOPOLÓGICAS:\n")
        f.write("-"*70 + "\n")
        if single_summary['has_transmembrane']:
            f.write("• Evidência de regiões transmembrana presentes.\n")
        else:
            f.write("• Não há evidência clara de regiões transmembrana.\n")
        if single_summary['has_signal_peptide']:
            f.write("• Evidência de peptídeo sinal presente.\n")
        else:
            f.write("• Não há evidência clara de peptídeo sinal.\n")

        f.write("\nNOTAS:\n")
        f.write("-"*70 + "\n")
        f.write("• Alta confiança (≥5 bancos) sugere função bem estabelecida.\n")
        f.write("• Baixa confiança (1–2 bancos) indica anotações mais incertas.\n")
        f.write("• Ausência de domínios não implica ausência de função, mas pode indicar proteínas específicas do organismo ou pouco caracterizadas.\n")

        f.write("="*70 + "\n")
        f.write(f"Relatório gerado em: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("="*70 + "\n")

    print(f"✅ Relatório salvo em: {report_path}")
else:
    print("❌ Não há dados suficientes para gerar o relatório. Execute as seções 3, 4 e 5 primeiro.")